# Intelligent Element Design

The idea of the intelligent element is to automate the creation of Keras models and data generation for complex nested structures.

In [1]:
from keras import Model
import keras.layers as L

def identity_fcn(x):
    return x

class IntelligentElement:
    '''
    Implements IntelligentElement, a base class that can be used to automate Keras model creation
    for complex nested structures
    '''
    
    def __init__(self, data, model, input_shape, preprocess_function = None, children_ie = None, name=None):
        '''
        Initializes the IntelligentElement instance with its data, which must be a list.
        
        All data passed to children IntelligentElements should be a list with the same number of elements
        and matching information. If a dynamic axis is found (input_shape[k] = None for some k), the vectors are 
        padded with zeroes until they get to the max length
        
        data        - list of samples that will be handled by this IE. If there is no model, it can contain empty elements
        children_ie - IEs that handle nested structures
        
        preprocess_function - function that is applied to each element of data list in order 
                              to retrieve neural network ready data as numpy arrays
        
        model - None if there is no data associated, otherwise a Keras Model
        '''
        
        assert isinstance(data, list), 'data should be a list of samples to be handled by this IntelligentElement'
        
        if model is not None:
            assert isinstance(model, Model), 'model should be a Keras model'
            assert len(model.output_shape) == 2, 'model output shape should have length 2'

        if children_ie is not None:
            for c in children_ie:
                assert isinstance(c, IntelligentElement), 'children_ie must contain only IntelligentElement'
                assert len(c.data) == len(data), 'length of data vector must be the same for parents and children'
        
        if preprocess_function is None:
            self.preprocess_function = identity_fcn
        else:
            self.preprocess_function = preprocess_function
            
        self.name=name
        self.data = data
        self.input_shape = input_shape
        self.children_ie = children_ie
        self.model = model
        
        if model is not None:
            self.model.name = 'm_{}'.format(name)

    def retrieve_model_inputs_outputs(self):
        if self.model is not None:
            inp = L.Input(self.input_shape, name='inp_{}'.format(self.name))
            inps = [inp]
            outs = [self.model(inp)]
        else:
            inps=[]
            outs=[]
        
        if self.children_ie is not None:
            for c in self.children_ie:
                cmodel, cinp, cout = c.retrieve_model_inputs_outputs()
                inps += cinp
                outs.append(cout)
                
        if len(outs) > 1:
            o = L.Concatenate()(outs)
        else:
            o = outs[0]
        
        ret_model = Model(inputs = inps, outputs = o, name = self.name)
        
        return ret_model, inps, o
    
    def get_batch(self, indices):
        '''
        Retrieves a batch of data as requested in indices
        '''
        
        batch_data = [self.data[i] for i in indices]
        
        if self.model is not None:
            cur_inps = [self.preprocess_function(x) for x in batch_data]
            
            #if there are no dynamic axes, we are done. if not, we need to pad
            shapes = np.array([x.shape for x in cur_inps])
            maxshape = np.max(shapes,axis=0)
            padded_inp = np.zeros( (len(indices),*maxshape) )
            #print('Padded shape: {}'.format(padded_inp.shape))
            
            cur_n=0
            for vec in cur_inps:
                ss = ( (cur_n,) )
                for p in vec.shape:
                    ss += (slice(0,p),)
                padded_inp[ss] = vec
                cur_n+=1
            
            inps = [padded_inp]
        else:
            inps = []
            
        if self.children_ie is not None:
            for c in self.children_ie:
                cinp = c.get_batch(indices)
                inps += cinp
            
        return inps

Using TensorFlow backend.


# Sample synthesized use

In [2]:
import numpy as np
import pandas as pd
regions_id = {'countryside' : 0, 'central' : 1, 'innerside' : 2}
def genClientData():
    regions = ['countryside', 'central', 'innerside']
    
    ans = {'income' : np.random.randint(10,50), 'age' : np.random.randint(20,70), 
           'residence_region' : regions[np.random.randint(len(regions))] }
    
    ans['last_transactions'] = []
    nlast = np.random.randint(3,10)
    for i in range(nlast):
        ans['last_transactions'].append({'month' : i, 'value':np.random.randint(1,5)})
    
    ans['picture'] = np.ones((128,128,3))
    
    return ans

clientdata = [genClientData() for i in range(20)]

In [3]:
pd.DataFrame(clientdata).head()

,age,income,last_transactions,picture,residence_region
0,49,29,"[{'month': 0, 'value': 1}, {'month': 1, 'value...","[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",countryside
1,55,11,"[{'month': 0, 'value': 3}, {'month': 1, 'value...","[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",innerside
2,30,29,"[{'month': 0, 'value': 2}, {'month': 1, 'value...","[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",countryside
3,21,38,"[{'month': 0, 'value': 4}, {'month': 1, 'value...","[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",countryside
4,29,28,"[{'month': 0, 'value': 4}, {'month': 1, 'value...","[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",countryside


In [4]:
picture_data = [x['picture'] for x in clientdata]
picture_shape = (128,128,3)
inp=L.Input(picture_shape)
x=L.Conv2D(10,(3,3), activation='relu', padding='same')(inp)
x=L.Flatten()(x)
x=L.Dense(5,activation='relu')(x)
picture_model = Model(inputs=inp, outputs=x)
print('Original model')
picture_model.summary()

picture_ie = IntelligentElement(picture_data, picture_model, picture_shape, name='picture_ie')

m, ii, oo = picture_ie.retrieve_model_inputs_outputs()
print('\n\nRetrieved model')
m.summary()

Original model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 10)      280       
_________________________________________________________________
flatten_1 (Flatten)          (None, 163840)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 819205    
Total params: 819,485
Trainable params: 819,485
Non-trainable params: 0
_________________________________________________________________


Retrieved model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inp_picture_ie (InputLayer)  (None, 128, 128, 3)       0         
_____________________________________

In [5]:
picture_ie.get_batch([0,4,5,8,9])[0].shape

(5, 128, 128, 3)

In [6]:
transaction_data = [x['last_transactions'] for x in clientdata]
transaction_shape = (None, 3)

def transaction_preproc_function(x):
    #receives a dictionary with keys 'month' and 'value'
    #append a 1 to indicate that these values were not padded
    ans = []
    for item in x:
        ans.append([item['month']/12, item['value']/1000, 1])
    
    #handle case when list is empty
    if len(ans) == 0:
        ans.append([0,0,0])
    
    return np.array(ans)


inp = L.Input(transaction_shape)
x = L.LSTM(64)(inp)
x = L.Dense(16, activation='relu')(x)
x = L.Dense(8, activation='relu')(x)
transaction_model = Model(inputs=inp, outputs=x)
print('Original model')
transaction_model.summary()

transaction_ie = IntelligentElement(transaction_data, transaction_model, transaction_shape, 
                                    preprocess_function=transaction_preproc_function, name='transaction_ie')

m, ii, oo = transaction_ie.retrieve_model_inputs_outputs()
print('\n\nRetrieved model')
m.summary()

Original model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, 3)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                17408     
_________________________________________________________________
dense_2 (Dense)              (None, 16)                1040      
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 136       
Total params: 18,584
Trainable params: 18,584
Non-trainable params: 0
_________________________________________________________________


Retrieved model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inp_transaction_ie (InputLay (None, None, 3)           0         
_______________________________________

In [7]:
transaction_ie.preprocess_function(transaction_ie.data[2])

array([[0.        , 0.002     , 1.        ],
       [0.08333333, 0.001     , 1.        ],
       [0.16666667, 0.001     , 1.        ]])

In [8]:
transaction_ie.get_batch([0,4,2,8,9])[0][2]

array([[0.        , 0.002     , 1.        ],
       [0.08333333, 0.001     , 1.        ],
       [0.16666667, 0.001     , 1.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ]])

In [9]:
#numerical for age and income, one-hot encoding for residence_region
client_data = [[x['age'], x['income'], x['residence_region']] for x in clientdata]
client_shape = (5,)
def client_preproc_function(x):
    #receives a list with age, income and residence_region
    idx = regions_id[x[2]]
    region_1hot = [0] * len(regions_id)
    region_1hot[idx] = 1

    return np.array([x[0]/100, np.log(1+x[1])]+region_1hot)

inp=L.Input(client_shape)
x=inp
x=L.Dense(8, activation='relu')(x)
x=L.Dense(8, activation='relu')(x)
client_model = Model(inputs=inp, outputs=x)

print('Original model')
client_model.summary()

client_ie = IntelligentElement(client_data, client_model, client_shape, preprocess_function=client_preproc_function,
                               children_ie=[picture_ie, transaction_ie], name='client_ie')

m, ii, oo = client_ie.retrieve_model_inputs_outputs()
print('\n\nRetrieved model')
m.summary()

Original model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 5)                 0         
_________________________________________________________________
dense_4 (Dense)              (None, 8)                 48        
_________________________________________________________________
dense_5 (Dense)              (None, 8)                 72        
Total params: 120
Trainable params: 120
Non-trainable params: 0
_________________________________________________________________


Retrieved model
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inp_client_ie (InputLayer)      (None, 5)            0                                            
______________________________________________________________________________

In [10]:
client_ie.preprocess_function(client_ie.data[2])

array([0.3       , 3.40119738, 1.        , 0.        , 0.        ])

In [11]:
b = client_ie.get_batch([0,4,2,8,9])
print(b[0])
print(b[2][2])

[[0.49       3.40119738 1.         0.         0.        ]
 [0.29       3.36729583 1.         0.         0.        ]
 [0.3        3.40119738 1.         0.         0.        ]
 [0.35       3.8286414  0.         1.         0.        ]
 [0.21       3.36729583 1.         0.         0.        ]]
[[0.         0.002      1.        ]
 [0.08333333 0.001      1.        ]
 [0.16666667 0.001      1.        ]
 [0.         0.         0.        ]
 [0.         0.         0.        ]
 [0.         0.         0.        ]
 [0.         0.         0.        ]
 [0.         0.         0.        ]]
